In [1]:
import numpy as np
import torch
from pathlib import Path
import lmdb
import torchaudio
import io
import pickle
from collections import defaultdict

In [2]:
lmdb_dir = '/mnt/data2/ketan/DiffPoseTalk/datasets/HDTF_TFHP-lmdb'
lmdb_dir = "/mnt/data2/anirudh/3DLipSync/diffposetalk/tfhp_hdtf_dataset"
lmdb_dir = Path(lmdb_dir)

In [3]:
coef_stats = dict(np.load(f"{lmdb_dir}/stats_train.npz"))


In [4]:
for key in list(coef_stats.keys()):
    print(f" {key}: {coef_stats[key].shape}")

 shape_mean: (100,)
 shape_std: (100,)
 shape_max: (100,)
 shape_min: (100,)
 exp_mean: (50,)
 exp_std: (50,)
 exp_max: (50,)
 exp_min: (50,)
 pose_mean: (6,)
 pose_std: (6,)
 pose_max: (6,)
 pose_min: (6,)
 pose_rot6d_mean: (12,)
 pose_rot6d_std: (12,)
 pose_rot6d_max: (12,)
 pose_rot6d_min: (12,)


In [5]:
coef_stats = {x: torch.tensor(coef_stats[x]) for x in coef_stats}

In [6]:
coef_fps = 25
audio_unit = 16000. / coef_fps
n_motions = 100
n_audio_samples = round(audio_unit * n_motions)
coef_total_len = n_motions * 2
audio_total_len = round(audio_unit * coef_total_len)

crop_strategy = 'random'
rot_representation = 'aa'

In [7]:
entries = []
split_file = f"{lmdb_dir}/train.txt"
with open(split_file, 'r') as f:
    for line in f:
        entries.append(line.strip())

In [8]:
lmdb_env = lmdb.open(str(lmdb_dir), readonly=True, lock=False, readahead=False, meminit=False)
with lmdb_env.begin(write=False) as txn:
    clip_len = pickle.loads(txn.get('metadata'.encode()))['seg_len']
    audio_clip_len = round(audio_unit * clip_len)
    print(audio_unit * clip_len)
    print(audio_clip_len)

64000.0
64000


In [9]:
# Start a read-only transaction
with lmdb_env.begin(write=False) as txn:
    # Create a cursor to iterate over all key-value pairs
    with txn.cursor() as cursor:
        # Iterate through all keys
        keys = [key.decode('utf-8') for key in cursor.iternext(keys=True, values=False)]
        
        # Print or return the keys
        for key in keys:
            print(key)

RD_AmandaStuck/000/000
RD_AmandaStuck/000/001
RD_AmandaStuck/000/002
RD_AmandaStuck/000/003
RD_AmandaStuck/000/004
RD_AmandaStuck/000/005
RD_AmandaStuck/000/006
RD_AmandaStuck/000/007
RD_AmandaStuck/000/008
RD_AmandaStuck/000/metadata
RD_AmandaStuck/001/000
RD_AmandaStuck/001/001
RD_AmandaStuck/001/002
RD_AmandaStuck/001/003
RD_AmandaStuck/001/004
RD_AmandaStuck/001/005
RD_AmandaStuck/001/006
RD_AmandaStuck/001/007
RD_AmandaStuck/001/008
RD_AmandaStuck/001/009
RD_AmandaStuck/001/010
RD_AmandaStuck/001/011
RD_AmandaStuck/001/012
RD_AmandaStuck/001/013
RD_AmandaStuck/001/014
RD_AmandaStuck/001/015
RD_AmandaStuck/001/016
RD_AmandaStuck/001/017
RD_AmandaStuck/001/018
RD_AmandaStuck/001/019
RD_AmandaStuck/001/020
RD_AmandaStuck/001/021
RD_AmandaStuck/001/022
RD_AmandaStuck/001/023
RD_AmandaStuck/001/024
RD_AmandaStuck/001/025
RD_AmandaStuck/001/026
RD_AmandaStuck/001/027
RD_AmandaStuck/001/028
RD_AmandaStuck/001/029
RD_AmandaStuck/001/030
RD_AmandaStuck/001/metadata
RD_AndyJorgensen/000/000

In [11]:
index=0
with lmdb_env.begin(write=False) as txn:
    meta_key = f'{entries[index]}/metadata'.encode()
    metadata = pickle.loads(txn.get(meta_key))
    seq_len = metadata['n_frames']

print(seq_len)

747


In [12]:
start_frame = np.random.randint(0, seq_len - coef_total_len + 1)
coef_keys = ['shape', 'exp', 'pose']
coef_dict = {k: [] for k in coef_keys}
audio = []
print(start_frame)

546


In [13]:
start_clip = start_frame // clip_len
end_clip = (start_frame + coef_total_len - 1) // clip_len + 1

5 8


In [14]:
print(start_clip)
print(end_clip)

5
8


In [14]:
with lmdb_env.begin(write=False) as txn:
    for clip_idx in range(start_clip, end_clip):
        key = f'{entries[index]}/{clip_idx:03d}'.encode()
        start_idx = max(start_frame - clip_idx * clip_len, 0)
        end_idx = min(start_frame + coef_total_len - clip_idx * clip_len, clip_len)
        print(f"key: {key}")
        print(f"start_idx: {start_idx}")
        print(f"end_idx: {end_idx}")
        
        entry = pickle.loads(txn.get(key))
        for coef_key in coef_keys:
            coef_dict[coef_key].append(entry['coef'][coef_key][start_idx:end_idx])

        audio_data = entry['audio']
        audio_clip, sr = torchaudio.load(io.BytesIO(audio_data))
        assert sr == 16000, f'Invalid sampling rate: {sr}'
        audio_clip = audio_clip.squeeze()
        audio.append(audio_clip[round(start_idx * audio_unit):round(end_idx * audio_unit)])

key: b'RD_BethMeyers/000/000'
start_idx: 21
end_idx: 100
key: b'RD_BethMeyers/000/001'
start_idx: 0
end_idx: 100
key: b'RD_BethMeyers/000/002'
start_idx: 0
end_idx: 21


In [15]:
print(len(coef_dict['shape']))
print(coef_dict['shape'][0].shape)
print(coef_dict['exp'][0].shape)
print(coef_dict['pose'][0].shape)

3
(79, 100)
(79, 50)
(79, 6)


In [16]:
coef_dict = {k: torch.tensor(np.concatenate(coef_dict[k], axis=0)) for k in coef_keys}

In [17]:
coef_dict['shape'].shape

torch.Size([200, 100])

In [18]:
audio = torch.cat(audio, dim=0)

In [19]:
audio.shape

torch.Size([128000])

In [20]:
audio_mean = audio.mean()
audio_std = audio.std()
audio = (audio - audio_mean) / (audio_std + 1e-5)

In [21]:
keys = ['shape', 'exp', 'pose']
coef_dict = {k: (coef_dict[k] - coef_stats[f'{k}_mean']) / (coef_stats[f'{k}_std'] + 1e-9) for k in keys}

In [22]:
# Extract two consecutive audio/coef clips
audio_pair = [audio[:n_audio_samples].clone(), audio[-n_audio_samples:].clone()]
coef_pair = [{k: coef_dict[k][:n_motions].clone() for k in keys},
                {k: coef_dict[k][-n_motions:].clone() for k in keys}]

In [25]:
coef_pair[0]['exp'].shape

torch.Size([100, 50])

In [26]:
## Style Dataset

In [28]:
lmdb_dir = '/mnt/data2/ketan/DiffPoseTalk/datasets/HDTF_TFHP-lmdb'
lmdb_dir = Path(lmdb_dir)
coef_stats = dict(np.load(f"{lmdb_dir}/stats_train.npz"))
coef_stats = {x: torch.tensor(coef_stats[x]) for x in coef_stats}

In [29]:
coef_fps = 25
audio_unit = 16000. / coef_fps
n_motions = 100
n_audio_samples = round(audio_unit * n_motions)
coef_total_len = n_motions * 2.1
audio_total_len = round(audio_unit * coef_total_len)

crop_strategy = 'random'
rot_representation = 'aa'
no_head_pose = True

In [30]:
entries = defaultdict(list)
split_file = f"{lmdb_dir}/train.txt"
with open(split_file, 'r') as f:
    for line in f:
        person_id = line.strip().split('/')[0]
        self.entries[person_id].append(line.strip().split()[0])
self.person_ids = list(self.entries.keys())

210.0